In [1]:
!git clone https://github.com/GadGadGad/vietnamese-ocr
!cd vietnamese-ocr && pip install -r requirement.txt

Cloning into 'vietnamese-ocr'...
remote: Enumerating objects: 579, done.
remote: Counting objects: 100% (579/579), done.
remote: Compressing objects: 100% (430/430), done.
remote: Total 579 (delta 154), reused 557 (delta 140), pack-reused 0 (from 0)
Receiving objects: 100% (579/579), 16.07 MiB | 44.49 MiB/s, done.
Resolving deltas: 100% (154/154), done.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.8/294.8 kB 16.1 MB/s eta 0:00:00
  Attempting uninstall: einops
    Found existing installation: einops 0.8.0
    Uninstalling einops-0.8.0:
      Successfully uninstalled einops-0.8.0
  Attempting uninstall: opt-einsum
    Found existing installation: opt_einsum 3.4.0
    Uninstalling opt_einsum-3.4.0:
      Successfully uninstalled opt_einsum-3.4.0
ERROR: pip's dependency resolver does not currently take into account all the packag

In [2]:
%cd vietnamese-ocr

/kaggle/working/vietnamese-ocr


In [3]:
import os
import cv2
import json

import matplotlib.pyplot as plt

from PIL import Image
from tqdm.notebook import tqdm

from vietocr.vietocr.tool.predictor import Predictor
from vietocr.vietocr.tool.config import Cfg

from PaddleOCR import PaddleOCR, draw_ocr

In [4]:
FONT = './PaddleOCR/doc/fonts/latin.ttf'

def predict(recognitor, detector, img_path, save_path, padding=4, dpi=100):
    # Load image
    img = cv2.imread(img_path)

    # Text detection
    result = detector.ocr(img_path, cls=False, det=True, rec=False)
    result = result[:][:][0]

    # Filter Boxes
    boxes = []
    for line in result:
        boxes.append([[int(line[0][0]), int(line[0][1])], [int(line[2][0]), int(line[2][1])]])
    boxes = boxes[::-1]

    # Add padding to boxes
    padding = 4
    for box in boxes:
        box[0][0] = box[0][0] - padding
        box[0][1] = box[0][1] - padding
        box[1][0] = box[1][0] + padding
        box[1][1] = box[1][1] + padding

    # Text recognizion
    texts = []
    for box in boxes:
        cropped_image = img[box[0][1]:box[1][1], box[0][0]:box[1][0]]
        try:
            cropped_image = Image.fromarray(cropped_image)
        except:
            continue

        rec_result = recognitor.predict(cropped_image)

        text = rec_result#[0]

        texts.append(text)
        #print(text)

    # Convert boxes to draw
    def get_rectangle_points(x1, y1, x2, y2):
        x_tl = x1
        y_tl = y2
        x_br = x2
        y_br = y1
        return [(x1, y1), (x2, y1), (x2, y2), (x1, y2)]
    _boxes = [get_rectangle_points(boxe[0][0], boxe[0][1], boxe[1][0], boxe[1][1]) for boxe in boxes]

    # Draw boxes and texts
    img = draw_ocr(img, _boxes, texts, scores=None, font_path=FONT)


    # Save image
    img_name = img_path.split('/')[-1]
    cv2.imwrite(os.path.join(save_path, img_name), img)

    # Display image
    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    height, width, depth = img.shape

    # What size does the figure need to be in inches to fit the image?
    figsize = width / float(dpi), height / float(dpi)

    # Create a figure of the right size with one axes that takes up the full figure
    fig = plt.figure(figsize=figsize)
    ax = fig.add_axes([0, 0, 1, 1])

    # Hide spines, ticks, etc.
    ax.axis('off')

    # Display the image.
    ax.imshow(img, cmap='gray')

    plt.show()

    return boxes, texts

def display_image_in_actual_size(img_path, dpi=200):
  im_data = cv2.imread(img_path)

  height, width, depth = im_data.shape

  # What size does the figure need to be in inches to fit the image?
  figsize = width / float(dpi), height / float(dpi)

  # Create a figure of the right size with one axes that takes up the full figure
  fig = plt.figure(figsize=figsize)
  ax = fig.add_axes([0, 0, 1, 1])

  # Hide spines, ticks, etc.
  ax.axis('off')

  # Display the image.
  ax.imshow(im_data, cmap='gray')

  plt.show()

In [5]:
detector = PaddleOCR(use_angle_cls=False, lang="vi", use_gpu=True)

# Configure of VietOCR
config = Cfg.load_config_from_name('vgg_transformer')
# config = Cfg.load_config_from_file('vietocr/config.yml')
# config['weights'] = '/Users/bmd1905/Desktop/pretrain_ocr/vi00_vi01_transformer.pth'

config['cnn']['pretrained'] = True
config['predictor']['beamsearch'] = True
config['device'] = 'cuda:0'

recognitor = Predictor(config)

download https://paddleocr.bj.bcebos.com/PP-OCRv3/english/en_PP-OCRv3_det_infer.tar to /root/.paddleocr/whl/det/en/en_PP-OCRv3_det_infer/en_PP-OCRv3_det_infer.tar


100%|██████████| 4.00M/4.00M [00:16<00:00, 238kiB/s] 


download https://paddleocr.bj.bcebos.com/PP-OCRv3/multilingual/latin_PP-OCRv3_rec_infer.tar to /root/.paddleocr/whl/rec/latin/latin_PP-OCRv3_rec_infer/latin_PP-OCRv3_rec_infer.tar


100%|██████████| 10.2M/10.2M [00:16<00:00, 608kiB/s] 


download https://paddleocr.bj.bcebos.com/dygraph_v2.0/ch/ch_ppocr_mobile_v2.0_cls_infer.tar to /root/.paddleocr/whl/cls/ch_ppocr_mobile_v2.0_cls_infer/ch_ppocr_mobile_v2.0_cls_infer.tar


100%|██████████| 2.19M/2.19M [00:14<00:00, 149kiB/s]

[2025/01/17 02:42:44] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='/root/.paddleocr/whl/det/en/en_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='/root/.paddleocr/whl/rec/latin/latin_PP-OCRv3_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_batch_num=6, max_text_length=25, rec_char_dict_path='/kag


Downloading: "https://download.pytorch.org/models/vgg19_bn-c79401a0.pth" to /root/.cache/torch/hub/checkpoints/vgg19_bn-c79401a0.pth
100%|██████████| 548M/548M [00:07<00:00, 77.1MB/s]
18533it [00:08, 2086.86it/s]


In [6]:
def predict(recognitor, detector, img_path, padding=4):
    # Load image
    img = cv2.imread(img_path)
    if img is None:
        print(f"Error: Unable to load image at {img_path}")
        return []

    # Text detection
    result = detector.ocr(img_path, cls=False, det=True, rec=False)
    result = result[:][:][0]

    # Filter Boxes
    boxes = []
    for line in result:
        boxes.append([[int(line[0][0]), int(line[0][1])], [int(line[2][0]), int(line[2][1])]])
    boxes = boxes[::-1]

    # Add padding to boxes
    for box in boxes:
        box[0][0] = max(0, box[0][0] - padding)
        box[0][1] = max(0, box[0][1] - padding)
        box[1][0] = min(img.shape[1], box[1][0] + padding)
        box[1][1] = min(img.shape[0], box[1][1] + padding)

    # Text recognition
    texts = []
    for box in boxes:
        cropped_image = img[box[0][1]:box[1][1], box[0][0]:box[1][0]]
        if cropped_image.size == 0:
            print(f"Warning: Empty cropped image for box {box}")
            continue

        try:
            cropped_image = Image.fromarray(cropped_image)
        except Exception as e:
            print(f"Error converting cropped image to PIL format: {e}")
            continue

        try:
            rec_result = recognitor.predict(cropped_image)
            text = rec_result  # Assuming rec_result is the recognized text
            texts.append(text)
        except Exception as e:
            print(f"Error during text recognition: {e}")
            continue

    return texts

In [7]:
def perform_ocr_on_folder(image_folder, recognitor, detector, output_file="ocr_results.json"):
    ocr_results = {}
    i = 0

    # Ensure the image folder exists
    if not os.path.exists(image_folder):
        raise FileNotFoundError(f"The folder {image_folder} does not exist.")

    # Iterate over each image file in the folder
    for image_file in tqdm(os.listdir(image_folder)):
        i += 1
        image_path = os.path.join(image_folder, image_file)

        # Ensure it's a file (not a directory)
        if os.path.isfile(image_path):
            try:
                # Perform OCR
                result = predict(recognitor, detector, image_path, padding=4)
                text = " ".join(result)  # Combine lines of text
            except Exception as e:
                print(f"Error processing {image_path}: {e}")
                text = ""

            # Save the result in the dictionary
            ocr_results[image_path] = text.strip()

    # Save the results to a JSON file
    with open(output_file, "w", encoding="utf-8") as f:
        json.dump(ocr_results, f, ensure_ascii=False, indent=4)

    print(f"OCR results saved to {output_file}")
    return ocr_results


In [8]:
train_image_folder = '/kaggle/input/vimmsd/train-images'
train_output_file="train_ocr_cache.json"
perform_ocr_on_folder(train_image_folder, recognitor, detector, train_output_file)

  0%|          | 0/10805 [00:00<?, ?it/s]

OCR results saved to train_ocr_cache.json


{'/kaggle/input/vimmsd/train-images/98c78da23f651d72af4ab7100d0f826fde43704c8273e7c56c476c4ecda51682.jpg': '% E E T DORTMUND ĐÃ CÓ MÙA GIẢI RẤT HAY 11 NHƯNG CHỈ ĐŨ HAY ĐỂ KHIẾN CẦU THỨ BAYERN PHẢI XEM DIỆN THOẠI NHIỀU HƠN MÀ THÔI facebookcom/trollbongda youtube.com/trollbongdaofficial',
 '/kaggle/input/vimmsd/train-images/733ef474a9c5a33c867728388aaf3f7247c90b34a86dec7ebb4bede260275871.jpg': '',
 '/kaggle/input/vimmsd/train-images/9f32b0eee517243bf80a0b1f9c9e52a44af9589135360a3e25f656b508fda1ff.jpg': 'NẾU BUỘC 1 SƠI DÂY THỪNG DÀI DƯỚI ĐUÔI Q CỦA CHIẾC TRỰC THĂNG ĐANG BAY, SỢI DÂY THỪNG ĐÓ SẼ CHUYỂN ĐÔNG THẾ NÀO THEO CHIẾC TRỰC THĂNG? Physics Exam NẾU TRẢ LÀ ĐÚNG BẠN XỨNG ĐÁNG LÀ NHÀ VẬT LÝ HỌC SÁNH NGANG VỚI CẢ ALBERT EINSTEIN -))',
 '/kaggle/input/vimmsd/train-images/c83c7c3f5936299e3d9f20952042e6c4454d8105b54683426bdaad365ddb11ca.jpg': 'stercard PHÚT B7 REALO I BAYERN PHÚT 90-1: REAL 21 BAYERN CHỈ CÓ THỂ LÀ ĐỘI BÓNG HOÀNG GIA youtube.com/trollbongdaofficial facebookcom/trollbongda',


In [9]:
test_image_folder = '/kaggle/input/vimmsd/dev-images'
test_output_file="test_ocr_cache.json"
perform_ocr_on_folder(test_image_folder, recognitor, detector, test_output_file)

  0%|          | 0/1413 [00:00<?, ?it/s]

OCR results saved to test_ocr_cache.json


{'/kaggle/input/vimmsd/dev-images/2d06d8c77c741d001916199346cc112847e6bcf61b3dcebc51b2a09b03ebcf78.jpg': 'THÌ NHỚ ĐÓM CON MÀY Dại CÚP CÁI RA NGOÀI ĐƯỜNG Kẹo PHA GẶP KỆO CON CON] XUỐNG TAO',
 '/kaggle/input/vimmsd/dev-images/c981f23fc77cebd06ea872ea2c0ff6ec43a9d2517366ed19e1c03d6971142d5a.jpg': 'DÙNG GALAXY Z FOLD3 5G HỌC BÀI ĐÃ LẮM, ĐA NHIỆM LẠI CÓ BÚT S-PEN Nè/ ĐANG TRỜI NGHE THèm CÓ THU CŨ ĐỔI MỚI Ở THẾ GIỚI DI ĐỘNG ÁI GHÊ VẬY: ) thegioldidong THÔI CON NHÀ MÌNH NGHèo DÙNG TẠM CÁI TABLET CỦA BỐ HỌC BÀI CŨNG ĐƯỢC Nè KÉT KÉT E',
 '/kaggle/input/vimmsd/dev-images/342c9a8f91adeacde0f2c26dee3e6b86861b43e948d10bbf2ee1e2a0bab96c99.jpg': 'Cuộc bầu cử có ảnh hưởng thế nào? Các cuộc bầu cử giữa nhiệm kỳ thường là ?hàn thử biểu? đánh giá thành tích của một vị tổng thống Mỹ Troàngiam trong hai năm đầu tại Nhà Trắng, Do đó, đảng của tổng thống thường sẽ mất ghế trong bầu cử giữa nhiệm kỳ. Đây là điều khiến Tổng thống Biden và đảng Dân chủ lo ngại, nhất là khi tỷ lệ ủng hộ ông đã liên tục ở mức dướ

In [10]:
private_image_folder = '/kaggle/input/vimmsd/test-images'
private_output_file="private_ocr_cache.json"
perform_ocr_on_folder(private_image_folder, recognitor, detector, private_output_file)

  0%|          | 0/1504 [00:00<?, ?it/s]

OCR results saved to private_ocr_cache.json


{'/kaggle/input/vimmsd/test-images/3cf39586f606b05725d746603605d891c3a6d334116d558b9127489752839fb9.jpg': 'Bác sĩ Vén áo lên tôi tiêm cái nào /muhangxom/ Vén áo kiểu đấy thì chích cho sưng bụng luôn 259 080 1 ngày Thích Phản hồi',
 '/kaggle/input/vimmsd/test-images/13a5019ff17a2ab0c70b1712ab5b8a2e61356e1be2d9aa26766c6f8929fffc7d.jpg': 'if. Senior Member 45 30 minutes ago 2 Nằm ở bản năng. Con đực nào cũng giữ lại ít nhiều phần hoang dã từ xa xưa, nó sẽ phát tiết mạnh hoặc giảm lại tùy vào môi trường sống, giáo dục, rèn luyện. Phần con gắn liền phần người. Phát huy phần người hạn chế phần con để cuộc sống tốt hơn. Nói thế chứ tôi là vozer, tôi thích múi mít, nhiều hơn 1 thì càng tốt',
 '/kaggle/input/vimmsd/test-images/8012aeb49536661782e4133c27b71f7e97a12ae24f5afef937ac1adf2c255234.jpg': 'ĐIỀN TỪ VÀO CHỖ TRÔNG: MƯỢT MỸ XÃ TRỌI APP MYVIB THÌ CHỈ CÓ THỂ LÀ MƯỢT MÀ, VƯỢT TRỘI, HUHU CHỈ CÓ Sẻ LÀ THƯƠNG YÊU GIÚP ĐỖ MÌNH/ MƯỢT MÀ VƯỢT TRỘI GÌ CĂNG? ĐỢI ĐỊ TUI Ý TAO LÀ MÀY CHUYỂN KHOẢN TAO CH